In [ ]:
################################
# add values based on time coord
# thanks chat
################################

import numpy as np
import pandas as pd
import xarray as xr

# Create a time range
time = pd.date_range('2020-01-01', periods=10, freq='D')

# Create an initial xarray Dataset
ds = xr.Dataset(
    {
        'temperature': (['time'], np.random.rand(len(time)) * 10)  # Random temperature values
    },
    coords={'time': time}
)

# Define a function that depends on time
def temperature_trend(time):
    return 0.5 * (time - np.datetime64('2020-01-01')).astype('timedelta64[D]').astype(int)

# Apply this function to each time point
trend_values = temperature_trend(ds.time.values)

# Add the new values to the dataset
ds['temperature_trend'] = ('time', trend_values)

print(ds)

ds.close()

In [ ]:
#################################
# add values based on x coord
# should be similar to above code
# thanks chat
#################################

import numpy as np
import pandas as pd
import xarray as xr

# Create a time range
time = pd.date_range('2020-01-01', periods=10, freq='D')
# Create an x range
x = np.linspace(0, 10, 5)

# Create an initial xarray Dataset
ds = xr.Dataset(
    {
        'temperature': (['time', 'x'], np.random.rand(len(time), len(x)) * 10)  # Random temperature values
    },
    coords={'time': time, 'x': x}
)

# Define a function that depends on the x coordinate
def elevation(x):
    return 2 * x + 5

# Apply the function to the x coordinate
elevation_values = elevation(ds.x.values)

# Broadcast the elevation values to match the dimensions of the dataset
elevation_broadcasted = np.broadcast_to(elevation_values, (len(ds.time), len(ds.x)))

# Add the new values to the dataset
ds['elevation'] = (['time', 'x'], elevation_broadcasted)

print(ds)

ds.close()

In [ ]:
import numpy as np
import xarray as xr

def main():
    vector_file = "January/results/video_capture_2024-01-02_09-44-20_velocimetry_results.nc"

    ds = xr.open_dataset(vector_file)

    spac_loc = spacial_location_wrapper(ds.y.values)
    spac_loc = np.broadcast_to(spac_loc, (len(ds.time), len(ds.y)))

    ds['spacial_location'] = (['time', 'y'], spac_loc)

    print(ds)

def spacial_location(y, m, subdiv):
    if y < m + subdiv:
        return 0
    elif (y >= m + subdiv) and (y < m + 2 * subdiv):
        return 1
    elif (y >= m + subdiv) and (y < m + 3 * subdiv):
        return 2
    elif (y >= m + subdiv) and (y < m + 4 * subdiv):
        return 3
    else:
        return 4

def spacial_location_wrapper(y_vals):
    maximum = max(y_vals)
    minimum = min(y_vals)
    subdiv = (maximum - minimum) / 5

    spac_loc = []
    for y in y_vals:
        spac_loc.append(spacial_location(y, minimum, subdiv))
    return spac_loc

main()

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta

# vector_file = "January/results/video_capture_2024-01-02_09-44-20_velocimetry_results.nc"
vector_file = "nc_old/video_capture_2024-05-17_13-49-18_velocimetry_results.nc"
ds = xr.open_dataset(vector_file)

name = vector_file.split('/')[-1].rsplit('_', maxsplit=2)[0]
year, month, day = name.split('_')[2].split('-')
hour, minute, second = name.split('_')[3].split('-')

rounded_minute = 15 * round(int(minute) / 15)
minute = 0 if rounded_minute == 60 else rounded_minute

dt = datetime(int(year), int(month), int(day), int(hour), minute, int(second))
dt_utc = (dt + timedelta(hours=7)).replace(second=0)

# date = f"{dt_utc.year}-{dt_utc.month:02}-{dt_utc.day:02} {dt_utc.hour:02}:{dt_utc.minute:02}:00"
date = dt_utc.strftime('%Y-%m-%d %H:%M:%S')

df = pd.read_csv('LR_WaterLab_AA_SourceID_1_QC_0_Year_2024.csv')
df = df[['DateTimeUTC', 'Discharge_cms']]
df = df[df.Discharge_cms >= 0]
print(df)

print(date in df.DateTimeUTC.values)
if dt_utc in df.DateTimeUTC.values:
    discharge_cms = df[df.DateTimeUTC == date].Discharge_cms.values[0]
else:
    discharge_cms = np.nan

discharge = xr.DataArray(
    data=discharge_cms * np.ones_like(ds['time']),
    dims=['time'],
    coords={'time': ds['time']}
)
ds['sun_altitude'] = discharge
